## Practice: Decision tree from scratch (well, we hope!)

### Part 1
- create random list of 5 integers
 - create a function that will find the split values by
     - sorting the list from lowest to highest value
     - finding the midpoint for each consecutive pair of values
     - returning midpoints as a list
 - create new random list of 5 integers and test your split values function 

> wherever you see `list` you can always substitute with `ndarray` if more appropriate

In [1]:
import numpy as np
from numpy.random import default_rng

rnd_list = default_rng(110)
int_list = rnd_list.integers(low=5, high=50, size = 5)
int_list

array([31, 46, 33, 28, 15], dtype=int64)

In [2]:
def splits(x):
    splits_points = []
    x_sorted = sorted(x)
    for i in range(len(x)-1):
        mid_point = (x_sorted[i] + x_sorted[i+1])/2
        splits_points.append(mid_point)
    return splits_points

In [3]:
splits(int_list)

[21.5, 29.5, 32.0, 39.5]

### Part 2
 - create random array of integers in shape (5, 3)
 - create a function that iterates over the columns and calculates the split points for each column
 - return as a dictionary with column name as key and list of split points as value
 - verify by hand that you are getting the correct split points

In [4]:
int_twoDlist = rnd_list.integers(low=5, high=100, size = (5,3))

int_twoDlist

array([[72, 71, 52],
       [ 9, 27, 27],
       [88, 89, 11],
       [54, 71, 92],
       [14, 98, 47]], dtype=int64)

In [5]:
def splits_per_col(data):
    col_split_pts = {}
    ncols = data.shape[1]
    for c in range(ncols):
        split_pts = splits(data[:,c])
        col_split_pts['col_'+ str(c)] = split_pts
    return col_split_pts

In [6]:
splits_per_col(int_twoDlist)

{'col_0': [11.5, 34.0, 63.0, 80.0],
 'col_1': [49.0, 71.0, 80.0, 93.5],
 'col_2': [19.0, 37.0, 49.5, 72.0]}

### Part 3
 - create random array of integers in shape (5, 2)
 - use the first column as feature data X and the second column as regression target y
 - create a function that takes in this array and
     - calculates the MAE of the root node
     - calculates the split points
     - for each split point
         - split the data into two parts
         - calculates the weighted MAE after the split
     - returns the split point (and the new MAE) that reduces the MAE of the root node the most
     
> Note: this function could use other funtions that perform the individual steps

In [7]:
int_twoDlist = rnd_list.integers(low=5, high=100, size = (5,2))

int_twoDlist

array([[40, 54],
       [85, 30],
       [98, 11],
       [85, 65],
       [48, 51]], dtype=int64)

In [8]:
def split_data(x,y,split_pt):
    mask = x > split_pt
    anti_mask = x < split_pt
    x_true = x[mask]
    y_true = y[mask]
    x_false = x[anti_mask]
    y_false = y[anti_mask]
    return x_true, y_true, x_false , y_false

In [9]:
x_true, y_true, x_false , y_false =split_data(int_twoDlist[:,0],int_twoDlist[:,1], 70)

In [10]:
def MAE(y, y_true, y_false):
    y_true_hat = np.mean(y_true)
    y_false_hat = np.mean(y_false)
    y_hat = np.mean(y)
    
    old_mae = np.mean(np.absolute(y-y_hat))
    new_mae = len(y_true)/ len(y) * np.mean(np.absolute(y_true-y_true_hat)) + \
              len(y_false)/ len(y) * np.mean(np.absolute(y_false-y_false_hat)) 
    return old_mae, new_mae
    

In [11]:
MAE(int_twoDlist[:,1],y_true,y_false)

(17.36, 12.466666666666667)

In [12]:
def get_best_split(data):
    x = data[:,0]
    y = data[:,1]
    split_pts = splits(x)
    results = {}
    for point in split_pts:
        x_true, y_true, x_false , y_false = split_data(x,y, point)
        errors = MAE(y, y_true, y_false)
        results[point] = errors
    best_point = min(results,key=results.get)
    return results,best_point

In [13]:
get_best_split(int_twoDlist)

({44.0: (17.36, 15.0),
  66.5: (17.36, 12.466666666666667),
  85.0: (17.36, 0.6000000000000001),
  91.5: (17.36, 8.0)},
 85.0,
 (17.36, 0.6000000000000001))

### Part 4 (Optional)
 - Modify **Part 3** code so that it can handle more than 1 feature

In [19]:
def get_best_split_features(data):
    n_feature = data.shape[1] - 1 
    results = {}
    for feature_idx in range(n_feature):
        x = data[:,feature_idx]
        y = data[:,-1]
        split_pts = splits(x)
        for point in split_pts:
            x_true, y_true, x_false , y_false = split_data(x,y, point)
            errors = MAE(y, y_true, y_false)
            results[feature_idx,point] = errors
    best_point = min(results,key=results.get)
    return results,best_point

In [20]:
int_nDlist = rnd_list.integers(low=5, high=100, size = (5,5))

int_nDlist

array([[69, 59, 40,  8, 15],
       [ 9, 30, 62, 67, 88],
       [52, 45, 11, 57, 17],
       [22, 75, 45, 24, 88],
       [28, 94, 45, 88,  7]], dtype=int64)

In [21]:
get_best_split_features(int_nDlist)

({(0, 15.5): (36.0, 22.5),
  (0, 25.0): (36.0, 2.4),
  (0, 40.0): (36.0, 21.999999999999996),
  (0, 60.5): (36.0, 30.400000000000002),
  (1, 37.5): (36.0, 22.5),
  (1, 52.0): (36.0, 34.733333333333334),
  (1, 67.0): (36.0, 35.4),
  (1, 84.5): (36.0, 28.8),
  (2, 25.5): (36.0, 30.8),
  (2, 42.5): (36.0, 21.999999999999996),
  (2, 45.0): (36.0, 0.4),
  (2, 53.5): (36.0, 22.5),
  (3, 16.0): (36.0, 30.400000000000002),
  (3, 40.5): (36.0, 34.86666666666667),
  (3, 62.0): (36.0, 35.4),
  (3, 77.5): (36.0, 28.8)},
 (2, 45.0))